9lo1w2`
# Undersampling k-space: KKI model
Deep learning for undersampled MRI reconstruction
- **Author**: Georgia Kanli
- **Date**: 01/09/2023
- **Purpose**: 2D-Unet - KI 
    - The deep learning model includes 2 components: (1): Under-sampling correction model in k-space domain (2): artefacts correction model in image domain

# **1. Import libraries**

In [1]:
import tensorflow as tf
tf.__version__


2024-10-30 12:03:34.459313: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 12:03:34.480617: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-30 12:03:34.999530: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


'2.13.0'

In [2]:
# %reset -f
%matplotlib inline 
%load_ext autoreload
%autoreload 2


import json
import os
import numpy as np

from utils.utilsFun import open_json
from mainFolder.main import run
    

/home/gkanli/miniconda3/envs/py3.10-tensorflow/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
from tensorflow.python.client import device_lib

In [4]:
from tensorflow.python.keras import backend as K
print(K._get_available_gpus())

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2024-10-30 12:03:36.470757: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


['/device:GPU:0']
Num GPUs Available:  1


2024-10-30 12:03:36.483616: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-30 12:03:36.483703: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-30 12:03:36.484850: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

# Input parameters

In [6]:
# path_start = '/run/user/2045141/gvfs/smb-share:server=sharetransrad.crp-sante.healthnet.lu,share=transrad/Projects/PS-MRS/Scratch/Georgia/Code/Proposed_Model/20230901/'

json_path = 'configs.json'
configs = open_json(json_path)
with open(json_path, 'w') as jsonFile:
    json.dump(configs, jsonFile)
configs = open_json(json_path)

GRlevel = np.zeros((8,2))
GRlevel[:,0] = [20, 25, 25, 20, 20, 10, 10, 4]
GRlevel[:,1] = [60, 45, 35, 30, 20, 20, 10, 7]
RAlevel = np.zeros((8,2))
RAlevel[:,0] = [48, 45, 48, 25, 25, 10, 10, 4]
RAlevel[:,1] = [60, 40, 30, 35, 20, 20, 10, 7]
UNlevel = np.zeros((8,2))
UNlevel[:,0] = [45, 45, 45, 25, 25, 10, 10, 4]
UNlevel[:,1] = [60, 40, 30, 35, 20, 20, 10, 7]

level = GRlevel
path_results = 'results/'
path_orig_data =  "data/6000cases/" #"data/6000cases/"
path = path_orig_data 
configs.data_folder_id = path_orig_data
configs.under_sampling = "True"
configs.us_type = "gradient" # "gradient" "uniform" "semi-random" "mix"
configs.add_noise = "True"
configs.add_motion_artifact = "False"
configs.fKline = "False"

configs.num_epochs = 150 #150
configs.batch_size = 16
configs.ID = "Knet"
US_gradient = 6

with open(json_path, 'w') as jsonFile:
    json.dump(configs, jsonFile)


In [7]:
import gc
tf.random.set_seed(1)
    
print('********************************************************** Knet 0 ')
# K-net
configs.ks_domain = "True"
configs.normalization_option = 'False'
configs.model = "2DUnet-leakyReLU" # 2DUnet-leakyReLU, 2DUnet, 2DUnet-tanh or complex_cnn
configs.loss = "mse" # "mssim" "fmssim" "mssim"  "fmse"
configs.under_sampling_uniform_factor = level[US_gradient,0]
configs.under_sampling_low_fr = level[US_gradient,1]
configs.data_folder_id = path_orig_data
configs.output_dir_base = path_results     
with open(json_path, 'w') as jsonFile:
    json.dump(configs, jsonFile)
    
model_K = run(configs)

********************************************************** Knet 0 
data/6000cases/
Load partition
Load partition:data/6000cases/
{'test': ['2446-28836-3', '2446-28836-7', '2446-28836-10', '2446-28836-14', '2446-28836-15', '2446-28836-6', '2446-28836-8', '2446-28836-13', '2446-28836-4', '2446-28836-5', '2446-28836-9', '2446-28836-12', '2447-28987-11', '2447-28987-15', '2447-28987-3', '2447-28987-7', '2447-28987-2', '2447-28987-6', '2447-28987-10', '2447-28987-14', '2447-28987-4', '2447-28987-8', '2447-28987-12', '2447-28987-9', '2447-28987-13', '2447-28987-1', '2447-28987-5', '2455-28863-7', '2455-28863-13', '2455-28863-3', '2455-29286-5', '2455-29286-9', '2455-29286-8', '2455-29286-11', '2455-28863-6', '2455-28863-12', '2455-29286-4', '2455-29286-13', '2455-28863-8', '2455-28863-4', '2455-28863-10', '2455-29286-6', '2455-28863-5', '2455-28863-11', '2455-29286-3', '2455-29286-7', '2455-29286-12', '2455-28863-9', '2456-28861-12', '2456-28861-13', '2456-28861-6', '2456-28861-11', '2456-28

/home/gkanli/miniconda3/envs/py3.10-tensorflow/lib/python3.10/site-packages/tensorflow_addons/optimizers/rectified_adam.py:121: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


ModelGenerator: Create the callbacks_out ... 
results//Knet/model_per_epoch/
ModelGenerator: Training ... 
Epoch 1/150


2024-10-30 12:20:54.543364: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-10-30 12:20:54.917837: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-30 12:20:55.745883: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


624/625 [============================>.] - ETA: 0s - loss: 0.5616 - mse: 0.5213 - ssim: 0.0481

/home/gkanli/miniconda3/envs/py3.10-tensorflow/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


625/625 [==============================] - 28s 39ms/step - loss: 0.5612 - mse: 0.5209 - ssim: 0.0481 - val_loss: 0.3612 - val_mse: 0.3210 - val_ssim: 0.0874 - lr: 1.0000e-04
Epoch 2/150
625/625 [==============================] - 24s 39ms/step - loss: 0.2296 - mse: 0.1898 - ssim: 0.1254 - val_loss: 0.2389 - val_mse: 0.1995 - val_ssim: 0.1599 - lr: 1.0000e-04
Epoch 3/150
625/625 [==============================] - 24s 39ms/step - loss: 0.1856 - mse: 0.1468 - ssim: 0.1742 - val_loss: 0.2058 - val_mse: 0.1675 - val_ssim: 0.2023 - lr: 1.0000e-04
Epoch 4/150
625/625 [==============================] - 24s 39ms/step - loss: 0.1687 - mse: 0.1311 - ssim: 0.2217 - val_loss: 0.1892 - val_mse: 0.1522 - val_ssim: 0.2493 - lr: 1.0000e-04
Epoch 5/150
625/625 [==============================] - 24s 39ms/step - loss: 0.1578 - mse: 0.1215 - ssim: 0.2638 - val_loss: 0.1781 - val_mse: 0.1425 - val_ssim: 0.2788 - lr: 1.0000e-04
Epoch 6/150
625/625 [==============================] - 24s 39ms/step - loss: 0.148